# Demonstration notebook of converting variety of input files to HEAL variable level metadata (ie data dictionary)
This takes a specified input file and uses the healdatautils to export HEAL-formatted data dictionaries.
The data dictionary titles are inferred from the file names. 

> Note, currently there are a few fields that do not have descriptions so return 
validation failure warnings. 

Will demonstrate two ways to create a data dictionary via the healdata-util `vlmd` tool.

1. Via python
2. Via command line

## Via python

In [1]:
from pathlib import Path 
from healdata_utils.cli import convert_to_vlmd
import os 
import pandas as pd
import json
import shutil

from healdata_utils.cli import input_descriptions
from IPython.display import Markdown,display

In [2]:
def printdir(dirname):
    for d in Path(dirname).iterdir():
        print(d)
        if Path(d).is_dir():
            for _d in Path(d).iterdir():
                print(f"   {_d}")

In [12]:
# current file paths for proof of concept
demo_filepaths = {
    # first 20 records of NMHSS SAMHDA Public Use File 
    "sas7bdat":"data/example_nmhss_2019_first_20recs.sas7bdat", 
    # SPSS/Stata examples created from pyreadstat via notebooks/demos/scripts/example.py
    "dta":"data/example_pyreadstat_output.dta", 
    "sav":"data/example_pyreadstat_output.sav",
    # The demostration CSV data dictionary exported from UChicago Redcap instance
    "redcap.csv":"data/example_redcap_demo.redcap.csv",
    # Valid csv version of pyreadstat 
    "csv":"data/example_pyreadstat_output.csv"
}


In [4]:
import pyreadstat
df,meta = pyreadstat.read_dta(demo_files["dta"],user_missing=True)

In [5]:
# available inputs
display(Markdown("Available inputs (except por and sas7bdat example does not have encodings/missing vals):"))
display(Markdown("".join(["- "+ext+"\n" for ext in list(input_descriptions.keys())])))
display(Markdown("Change the variable `input_type` to one of the extensions "))

Available inputs (except por):

- csv
- sav
- dta
- por
- sas7bdat
- json
- redcap.csv


Change the variable `input_type` to one of the extensions 

In [11]:
input_type = "sas7bdat"

Description for selected input type below:

In [7]:
display(Markdown(f"Input type: **`{input_type}`**"))
display(Markdown((input_descriptions[input_type])))

Input type: **`sav`**

Converts a "metadata-rich" (ie statistical software file) 
    into a HEAL-specified data dictionary in both csv format and json format.

    This function relies on [readstat](https://github.com/Roche/pyreadstat) which supports SPSS (sav and por), 
    SAS (sas7bdat), and Stata (dta). 

    > Currently, this function uses both data and metadata to generate 
    a HEAL specified data dictionary. That is, types are inferred from the 
    data (so at least test or synthetic data needed) while everything else is taken 
    from the metadata (eg missing values, variable labels, variable value labels etc)

    

In [8]:
description = "This is a proof of concept to demonstrate the healdata-utils functionality"
title = "Healdata-utils Demonstration Data Dictionary"
healdir = "output"

In [9]:
# make python demo output
Path(healdir).mkdir(exist_ok=True)

In [13]:
data_dictionaries = convert_to_vlmd(
    filepath=demo_filepaths[input_type],
    outputdir=healdir, #if not specified, will not write to file
    inputtype=input_type, #if not specified, looks for suffix
    data_dictionary_props={
        "name":Path(inputpath).stem,
        "title":title,
        "description":description}
)

Validating csv data dictionary...
Csv is VALID
Validating heal-specified json fields.....
JSON array of data dictionary fields is VALID


In [39]:
Markdown("Here is the resulting contents of the file directory:")
print(printdir("output"))

output\errors
   output\errors\heal-csv-errors-summary.txt
   output\errors\heal-csv-errors.json
   output\errors\heal-json-errors.json
output\heal-csvtemplate-data-dictionary.csv
output\heal-jsontemplate-data-dictionary.json
None


Resulting CSV fields

Examine human-readable csv validation report. Say a data dictionary is not valid. The csv report summary will give these errors. If this is the case, you can edit the csv data dictionary and re-run `convert_vlmd` with the csv input type. For an example of this, see the csv validation demo notebook. In this notebook, all files are valid, so the summary will return a 
report indicating it is valid.

In [40]:
print(Path("output/errors/heal-csv-errors-summary.txt").read_text())


# -----
# valid: memory 
# -----

## Summary 

+------------------------+-------------------+
| Description            | Size/Name/Count   |
+========================+===================+
| File name (Not Found)  | memory            |
+------------------------+-------------------+
| File size              | N/A               |
+------------------------+-------------------+
| Total Time Taken (sec) | 0.02              |
+------------------------+-------------------+




You can view the data dictionary by looking via a pandas dataframe from the written file or directly from the returned
data dictionary object. 

In [41]:
pd.DataFrame(data_dictionaries['csvtemplate']).head()

,module,name,title,description,type,format,constraints.maxLength,constraints.enum,constraints.pattern,constraints.maximum,...,univar_stats.median,univar_stats.mean,univar_stats.std,univar_stats.min,univar_stats.max,univar_stats.mode,univar_stats.count,univar_stats.twenty_five_percentile,univar_stats.seventy_five_percentile,univar_stats.cat_marginals
0,,id,,Unique identifier for participant,integer,,,,,,...,,,,,,,,,,
1,,visit_dt,,Date of the interview,datetime,,,,,,...,,,,,,,,,,
2,,sex_at_birth,,The self-reported sex of the participant/subje...,integer,,,1|2|3|4|b|a,,,...,,,,,,,,,,
3,,race,,Self-reported race,integer,,,1|2|3|4|5|6|7|8|b|a,,,...,,,,,,,,,,
4,,hispanic_ethnicity,,"Are you of Hispanic, Latino, or Spanish origin?",integer,,,0|1|a|b,,,...,,,,,,,,,,


Resulting JSON object 

> Note how currently the fields are nested within the data_dictionary property) as opposed to the csv tempalte which just has fields.

In [42]:
print(json.dumps(data_dictionaries['jsontemplate'],indent=4)[0:1000])

{
    "name": "example_pyreadstat_output",
    "title": "Healdata-utils Demonstration Data Dictionary",
    "description": "This is a proof of concept to demonstrate the healdata-utils functionality",
    "data_dictionary": {
        "name": "example_pyreadstat_output",
        "title": "Healdata-utils Demonstration Data Dictionary",
        "description": "This is a proof of concept to demonstrate the healdata-utils functionality",
        "data_dictionary": [
            {
                "name": "id",
                "type": "integer",
                "description": "Unique identifier for participant"
            },
            {
                "name": "visit_dt",
                "type": "datetime",
                "description": "Date of the interview"
            },
            {
                "name": "sex_at_birth",
                "type": "integer",
                "encodings": {
                    "1": "Male",
                    "2": "Female",
                    "3": "Int

## Via command line

We will demonstrate the `vlmd` command line utility using one of the data dictionaries. 

In [43]:
# make a separate output-cli folder for cli demo

Path("output-cli").mkdir(exist_ok=True)

In [44]:
!vlmd --help

Usage: vlmd [OPTIONS]

Options:
  --filepath TEXT                 Path to the file you want to convert to a
                                  HEAL data dictionary  [required]
  --title TEXT                    The title of your data dictionary. If not
                                  specified, then the file name will be used
  --description TEXT              Description of data dictionary
  --inputtype [csv|sav|dta|por|sas7bdat|json|redcap.csv]
                                  The type of your input file.
  --outputdir TEXT                The folder where you want to output your
                                  HEAL data dictionary
  --help                          Show this message and exit.


To create the above data dictionary via the command line, run directly in this notebook with the cell below:

In [28]:
!vlmd --filepath "data/example_pyreadstat_output.sav" \
--outputdir "output-cli" \
--title "Healdata-utils Demonstration Data Dictionary" \
--description "This is a proof of concept to demonstrate the healdata-utils functionality" 

Validating csv data dictionary...
Csv is VALID
Validating heal-specified json fields.....
JSON array of data dictionary fields is VALID


In [29]:
printdir("output-cli")

output-cli\errors
   output-cli\errors\heal-csv-errors-summary.txt
   output-cli\errors\heal-csv-errors.json
   output-cli\errors\heal-json-errors.json
output-cli\heal-csvtemplate-data-dictionary.csv
output-cli\heal-jsontemplate-data-dictionary.json
